Transacciones ACID (Insert/ Update / Delete) en Hive
===

* *30 min* | Última modificación: Junio 22, 2019

El lenguaje SQL estándar provee directivas para la insertar, actualizar y borrar registros en una tabla. En este tutorial se presentan ejemplos representativos de estas instrucciones en Hive. 

In [1]:
%load_ext bigdata
%timeout 300

## Creación de la tabla

In [2]:
%%hive
DROP DATABASE IF EXISTS demo CASCADE;
CREATE DATABASE demo;
USE demo;

CREATE TABLE persons (
    id        INT,
    firstname STRING,
    surname   STRING,
    birthday  TIMESTAMP,
    quantity  INT
)
PARTITIONED BY (color STRING)
CLUSTERED BY(id) INTO 3 BUCKETS
STORED AS ORC 
LOCATION '/tmp/hive-partitioned'
TBLPROPERTIES ('transactional'='true');

DROP DATABASE IF EXISTS demo CASCADE;
OK
Time taken: 8.526 seconds
CREATE DATABASE demo;
OK
Time taken: 0.49 seconds
USE demo;
OK
Time taken: 0.011 seconds
CREATE TABLE persons (
    id        INT,
    firstname STRING,
    surname   STRING,
    birthday  TIMESTAMP,
    quantity  INT
)
PARTITIONED BY (color STRING)
CLUSTERED BY(id) INTO 3 BUCKETS
STORED AS ORC 
LOCATION '/tmp/hive-partitioned'
TBLPROPERTIES ('transactional'='true');
OK
Time taken: 0.539 seconds


## Preparación

Se deben habilitar las características de Hive para manejo de transacciones ACID.

In [3]:
%%hive
SET hive.txn.manager=org.apache.hadoop.hive.ql.lockmgr.DbTxnManager;
SET hive.support.concurrency=true;
SET hive.enforce.bucketing=true;
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.compactor.initiator.on=true;
SET hive.compactor.worker.threads=1;

SET hive.txn.manager=org.apache.hadoop.hive.ql.lockmgr.DbTxnManager;
SET hive.support.concurrency=true;
SET hive.enforce.bucketing=true;
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.compactor.initiator.on=true;
SET hive.compactor.worker.threads=1;


---

## INSERT

    INSERT INTO TABLE tablename VALUES PARTITION () values_row [, values_row ...]
    
    values_row: 
       (value [, value ...])
       
Note que a diferencia de SQL, aca no es posible indicar para que columnas se van a insertar los valores, de tal manera que siempre se deben dar valores para todas las columnas.       

In [4]:
%%hive
--
-- Inserta el registro en la tabla.
-- Los valores están en el mismo orden de los campos.
--
INSERT INTO persons PARTITION (color='green') VALUES
    (1,"Vivian","Hamilton","1971-07-08",1),
    (2,"Karen","Holcomb","1974-05-23",4),
    (12,"Hope","Coffey","1973-12-24",5),
    (17,"Chanda","Boyer","1973-04-01",4);
    
INSERT INTO persons PARTITION (color='black') VALUES    
    (4,"Roth","Fry","1975-01-29",1),
    (10,"Kylan","Sexton","1975-02-28",4);

INSERT INTO persons PARTITION (color='blue') VALUES
    (5,"Zoe","Conway","1974-07-03",2),
    (7,"Driscoll","Klein","1970-10-05",5),
    (15,"Hope","Silva","1970-07-01",5);

INSERT INTO persons PARTITION (color='orange') VALUES    
    (3,"Cody","Garrett","1973-04-22",1),
    (16,"Ayanna","Jarvis","1974-02-11",5);
    
INSERT INTO persons PARTITION (color='violet') VALUES    
    (6,"Gretchen","Kinney","1974-10-18",1);

INSERT INTO persons PARTITION (color='red') VALUES    
    (8,"Karyn","Diaz","1969-02-24",1),
    (14,"Clio","Noel","1972-12-12",5);
    
INSERT INTO persons PARTITION (color='indigo') VALUES    
    (9,"Merritt","Guy","1974-10-17",4),
    (11,"Jordan","Estes","1969-12-07",4);

INSERT INTO persons PARTITION (color='gray') VALUES    
    (13,"Vivian","Crane","1970-08-27",5);

INSERT INTO persons PARTITION (color='yellow') VALUES
    (18,"Chadwick","Knight","1973-04-29",1);    


--
-- Inserta el registro en la tabla.
-- Los valores están en el mismo orden de los campos.
--
INSERT INTO persons PARTITION (color='green') VALUES
    (1,"Vivian","Hamilton","1971-07-08",1),
    (2,"Karen","Holcomb","1974-05-23",4),
    (12,"Hope","Coffey","1973-12-24",5),
    (17,"Chanda","Boyer","1973-04-01",4);
Query ID = vagrant_20190607124023_1702031a-ab20-4e2d-9aa6-e71950b914aa
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 3
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1558824450495_0111, Tracking URL = http://ubuntu-bionic:8088/proxy/application_1558824450495_0111/
Kill Command = /usr/local/hadoop-2.8.5/bin/hadoop job  -kill job_1558824450495_0111
Hadoop job information for Sta

In [5]:
%%hive
SELECT * FROM persons;

SELECT * FROM persons;
OK
10	Kylan	Sexton	1975-02-28 00:00:00	4	black
4	Roth	Fry	1975-01-29 00:00:00	1	black
15	Hope	Silva	1970-07-01 00:00:00	5	blue
7	Driscoll	Klein	1970-10-05 00:00:00	5	blue
5	Zoe	Conway	1974-07-03 00:00:00	2	blue
13	Vivian	Crane	1970-08-27 00:00:00	5	gray
12	Hope	Coffey	1973-12-24 00:00:00	5	green
1	Vivian	Hamilton	1971-07-08 00:00:00	1	green
17	Chanda	Boyer	1973-04-01 00:00:00	4	green
2	Karen	Holcomb	1974-05-23 00:00:00	4	green
9	Merritt	Guy	1974-10-17 00:00:00	4	indigo
11	Jordan	Estes	1969-12-07 00:00:00	4	indigo
3	Cody	Garrett	1973-04-22 00:00:00	1	orange
16	Ayanna	Jarvis	1974-02-11 00:00:00	5	orange
14	Clio	Noel	1972-12-12 00:00:00	5	red
8	Karyn	Diaz	1969-02-24 00:00:00	1	red
6	Gretchen	Kinney	1974-10-18 00:00:00	1	violet
18	Chadwick	Knight	1973-04-29 00:00:00	1	yellow
Time taken: 0.292 seconds, Fetched: 18 row(s)


In [6]:
!hdfs dfs -ls /tmp/hive-partitioned

Found 9 items
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:41 /tmp/hive-partitioned/color=black
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:41 /tmp/hive-partitioned/color=blue
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:43 /tmp/hive-partitioned/color=gray
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:40 /tmp/hive-partitioned/color=green
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:43 /tmp/hive-partitioned/color=indigo
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:41 /tmp/hive-partitioned/color=orange
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:42 /tmp/hive-partitioned/color=red
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:42 /tmp/hive-partitioned/color=violet
drwxrwxrwx   - vagrant supergroup          0 2019-06-07 12:44 /tmp/hive-partitioned/color=yellow


In [7]:
!hdfs dfs -ls /tmp/hive-partitioned/color=black

Found 1 items
drwxr-xr-x   - vagrant supergroup          0 2019-06-07 12:41 /tmp/hive-partitioned/color=black/delta_0000026_0000026_0000


In [9]:
!hdfs dfs -ls /tmp/hive-partitioned/color=black/delta_0000026_0000026_0000

Found 3 items
-rw-r--r--   1 vagrant supergroup        223 2019-06-07 12:41 /tmp/hive-partitioned/color=black/delta_0000026_0000026_0000/bucket_00000
-rw-r--r--   1 vagrant supergroup        940 2019-06-07 12:41 /tmp/hive-partitioned/color=black/delta_0000026_0000026_0000/bucket_00001
-rw-r--r--   1 vagrant supergroup        223 2019-06-07 12:41 /tmp/hive-partitioned/color=black/delta_0000026_0000026_0000/bucket_00002


## UPDATE

    UPDATE tablename SET column = value [, column = value ...] [WHERE expression]

Véase https://cwiki.apache.org/confluence/display/Hive/Hive+Transactions

In [12]:
%%hive
UPDATE persons SET quantity = 100 WHERE color = 'red';

UPDATE persons SET quantity = 100 WHERE color = 'red';
Query ID = vagrant_20190607124609_a78bda28-0719-40d5-aeae-02076a67d2c6
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 3
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1558824450495_0120, Tracking URL = http://ubuntu-bionic:8088/proxy/application_1558824450495_0120/
Kill Command = /usr/local/hadoop-2.8.5/bin/hadoop job  -kill job_1558824450495_0120
Hadoop job information for Stage-1: number of mappers: 3; number of reducers: 3
2019-06-07 12:46:14,724 Stage-1 map = 0%,  reduce = 0%
2019-06-07 12:46:20,153 Stage-1 map = 33%,  reduce = 0%, Cumulative CPU 1.74 sec
2019-06-07 12:46:21,209 Stage-1 map = 67%,  reduce = 0%, Cumulative CPU 3.38 

In [13]:
%%hive
SELECT * FROM persons;

SELECT * FROM persons;
OK
10	Kylan	Sexton	1975-02-28 00:00:00	4	black
4	Roth	Fry	1975-01-29 00:00:00	1	black
15	Hope	Silva	1970-07-01 00:00:00	5	blue
7	Driscoll	Klein	1970-10-05 00:00:00	5	blue
5	Zoe	Conway	1974-07-03 00:00:00	2	blue
13	Vivian	Crane	1970-08-27 00:00:00	5	gray
12	Hope	Coffey	1973-12-24 00:00:00	5	green
1	Vivian	Hamilton	1971-07-08 00:00:00	1	green
17	Chanda	Boyer	1973-04-01 00:00:00	4	green
2	Karen	Holcomb	1974-05-23 00:00:00	4	green
9	Merritt	Guy	1974-10-17 00:00:00	4	indigo
11	Jordan	Estes	1969-12-07 00:00:00	4	indigo
3	Cody	Garrett	1973-04-22 00:00:00	1	orange
16	Ayanna	Jarvis	1974-02-11 00:00:00	5	orange
14	Clio	Noel	1972-12-12 00:00:00	100	red
8	Karyn	Diaz	1969-02-24 00:00:00	100	red
6	Gretchen	Kinney	1974-10-18 00:00:00	1	violet
18	Chadwick	Knight	1973-04-29 00:00:00	1	yellow
Time taken: 0.168 seconds, Fetched: 18 row(s)


## DELETE

    DELETE FROM tablename [WHERE expression]
    
Véase https://cwiki.apache.org/confluence/display/Hive/Hive+Transactions    

In [14]:
%%hive
DELETE FROM persons WHERE id = 10;

DELETE FROM persons WHERE id = 10;
Query ID = vagrant_20190607124655_41039dd6-fdf7-467d-bd81-ddd51a02037f
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 3
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1558824450495_0121, Tracking URL = http://ubuntu-bionic:8088/proxy/application_1558824450495_0121/
Kill Command = /usr/local/hadoop-2.8.5/bin/hadoop job  -kill job_1558824450495_0121
Hadoop job information for Stage-1: number of mappers: 27; number of reducers: 3
2019-06-07 12:47:00,892 Stage-1 map = 0%,  reduce = 0%
2019-06-07 12:47:06,488 Stage-1 map = 4%,  reduce = 0%, Cumulative CPU 2.22 sec
2019-06-07 12:47:08,729 Stage-1 map = 7%,  reduce = 0%, Cumulative CPU 4.47 sec
2019-06-07 12:47:

In [15]:
%%hive
SELECT * FROM persons ORDER BY id;

SELECT * FROM persons ORDER BY id;
Query ID = vagrant_20190607124801_b7d15cb7-d826-4693-8eff-9714f6d585a2
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1558824450495_0122, Tracking URL = http://ubuntu-bionic:8088/proxy/application_1558824450495_0122/
Kill Command = /usr/local/hadoop-2.8.5/bin/hadoop job  -kill job_1558824450495_0122
Hadoop job information for Stage-1: number of mappers: 27; number of reducers: 1
2019-06-07 12:48:08,318 Stage-1 map = 0%,  reduce = 0%
2019-06-07 12:48:12,690 Stage-1 map = 4%,  reduce = 0%, Cumulative CPU 1.47 sec
2019-06-07 12:48:14,953 Stage-1 map = 7%,  reduce = 0%, Cumulative CPU 3.05 sec
2019-06-07 12:48:

## MERGE

    MERGE INTO <target table> AS T USING <source expression/table> AS S
    ON <boolean expression1>
    WHEN MATCHED [AND <boolean expression2>] THEN UPDATE SET <set clause list>
    WHEN MATCHED [AND <boolean expression3>] THEN DELETE
    WHEN NOT MATCHED [AND <boolean expression4>] THEN INSERT VALUES<value list>
    
Véase https://community.hortonworks.com/articles/97113/hive-acid-merge-by-example.html    

---

In [ ]:
%%hive
-- limpia la base de datos
DROP DATABASE IF EXISTS demo CASCADE;